### I have no idea what this data details and what kind of insights I could gather from this. Lets explore shall we

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Excel file has two sheets, facts and cities. Lets do some exploration

In [2]:
df = pd.ExcelFile('countries_cities_data.xlsx')
sheet_names = df.sheet_names
print(sheet_names)

['facts', 'cities']


In [3]:
df = pd.read_excel('countries_cities_data.xlsx', sheet_name=None)
df_facts = df['facts']
df_cities = df['cities']

print(df_facts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 261 non-null    int64  
 1   code               261 non-null    object 
 2   name               261 non-null    object 
 3   area               249 non-null    float64
 4   area_land          246 non-null    float64
 5   area_water         243 non-null    float64
 6   population         242 non-null    float64
 7   population_growth  236 non-null    float64
 8   birth_rate         228 non-null    float64
 9   death_rate         228 non-null    float64
 10  migration_rate     223 non-null    float64
dtypes: float64(8), int64(1), object(2)
memory usage: 22.6+ KB
None


In [4]:
print(df_cities.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          397 non-null    int64 
 1   name        397 non-null    object
 2   population  397 non-null    int64 
 3   capital     397 non-null    int64 
 4   facts_id    397 non-null    int64 
 5   gdp         397 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 18.7+ KB
None


In [5]:
print(df_facts.head())

   id code         name       area  area_land  area_water  population  \
0   1   af  Afghanistan   652230.0   652230.0         0.0  32564342.0   
1   2   al      Albania    28748.0    27398.0      1350.0   3029278.0   
2   3   ag      Algeria  2381741.0  2381741.0         0.0  39542166.0   
3   4   an      Andorra      468.0      468.0         0.0     85580.0   
4   5   ao       Angola  1246700.0  1246700.0         0.0  19625353.0   

   population_growth  birth_rate  death_rate  migration_rate  
0               2.32       38.57       13.89            1.51  
1               0.30       12.92        6.58            3.30  
2               1.84       23.67        4.31            0.92  
3               0.12        8.13        6.96            0.00  
4               2.78       38.78       11.49            0.46  


In [6]:
print(df_cities.head())

   id              name  population  capital  facts_id    gdp
0   6             Kabul     3097000        1         1  12273
1  10            Tirana      419000        1         2  13232
2   7           Algiers     2916000        1         3  12734
3   8              Oran      783000        0         3  12828
4  12  Andorra La Vella       23000        1         4   1675


### From the sort, i can tell that the ids of both sheets are not related (not a primary key), however it would seem that the facts_id in df_cities is the foreign key for df_facts.

In [7]:
df_cities.sort_values(by='id', ascending=True, inplace=True)
print(df_cities.head())

     id          name  population  capital  facts_id   gdp
382   1    Oranjestad       37000        1       216  5527
7     2  Saint John'S       27000        1         6  5549
344   3     Abu Dhabi      942000        1       184  4219
345   4         Dubai     1978000        0       184  6309
346   5       Sharjah      983000        0       184  6519


### No Null values

In [8]:
null_values = df_cities.isnull().sum()
print(null_values)

id            0
name          0
population    0
capital       0
facts_id      0
gdp           0
dtype: int64


### No duplicates

In [9]:
duplicates = df_cities.duplicated().sum()
print(duplicates)

0


### No population below 0 and data type is int

In [10]:
invalid_population = df_cities[~df_cities['population'].apply(lambda x: isinstance(x, int) and x >= 0)]
print(invalid_population)

Empty DataFrame
Columns: [id, name, population, capital, facts_id, gdp]
Index: []


### Probably shows if the city is a capital of the country (fact_id) or not. 
* 1 being yes and 0 being no

In [11]:
print(df_cities['capital'].value_counts())

capital
1    208
0    189
Name: count, dtype: int64


# Now we do the same for df_facts

### No duplicated values

In [12]:
duplicated_facts = df_facts.duplicated().sum()
print(duplicated_facts)

0


1. area being 0 is kinda odd.
2. area_land and area_water being 0 could jus mean that they have no land nor water (makes sense for water but land need to think abit more)
3. The rest being 0 does not really make sense
4. For population, we could get some details from the cities sheet.

In [13]:
null_facts = df_facts.isnull().sum()
print(null_facts)


id                    0
code                  0
name                  0
area                 12
area_land            15
area_water           18
population           19
population_growth    25
birth_rate           33
death_rate           33
migration_rate       38
dtype: int64


In [14]:
test_facts = df_facts.copy()
test_facts['area_test'] = test_facts['area_land'] + test_facts['area_water']
test_facts['test'] = test_facts['area_test'] == test_facts['area']
print(test_facts['test'].value_counts())

test
True     239
False     22
Name: count, dtype: int64


### The reason for some of the False values is because of mismatched typed. int + NaN = NaN. So i should had transformed NaN to int 0 or float first before doing the addition and checks

In [15]:
print(test_facts[test_facts['test'] == False])

      id code                                           name       area  \
34    35   cd                                           Chad        NaN   
57    58   et                                       Ethiopia  1104300.0   
125  126   nz                                    New Zealand   267710.0   
127  128   ng                                          Niger        NaN   
161  162   od                                    South Sudan   644329.0   
164  165   su                                          Sudan  1861484.0   
196  197   ee                                 European Union  4324782.0   
206  207   gl                                      Greenland  2166086.0   
209  210   fs            French Southern and Antarctic Lands        NaN   
211  212   tb                               Saint Barthelemy        NaN   
212  213   rn                                   Saint Martin       54.0   
224  225   ax                                       Akrotiri      123.0   
230  231   dx            

In [16]:
test_facts.drop(columns=['area_test', 'test'], inplace=True)
test_facts = test_facts.fillna(0.0)
test_facts['area_test'] = test_facts['area_land'] + test_facts['area_water']
test_facts['test'] = test_facts['area_test'] == test_facts['area']
print(test_facts['test'].value_counts())

test
True     251
False     10
Name: count, dtype: int64


### A few things are happening
1. area is 0 but there are area_land and area_water data
2. There is area value, and either area_land or area_water are 0, and area != either area_land/area_water values
3. There is area value and both area_land and area_water are 0, in this case I cant do anything about it 
### For points 1 and 2, I can do something about

In [17]:
print(test_facts[test_facts['test'] == False])

      id code                                           name       area  \
34    35   cd                                           Chad        0.0   
57    58   et                                       Ethiopia  1104300.0   
127  128   ng                                          Niger        0.0   
161  162   od                                    South Sudan   644329.0   
164  165   su                                          Sudan  1861484.0   
196  197   ee                                 European Union  4324782.0   
224  225   ax                                       Akrotiri      123.0   
230  231   dx                                       Dhekelia      130.0   
238  239   sh  Saint Helena, Ascension, and Tristan da Cunha      308.0   
249  250   ay                                     Antarctica        0.0   

     area_land  area_water   population  population_growth  birth_rate  \
34   1259200.0     24800.0   11631456.0               1.89       36.60   
57         0.0    104300.0

In [18]:
test_facts.loc[(test_facts['area'] == 0) & (test_facts['area_land'] != 0) & (
    test_facts['area_water'] != 0), 'area'] = test_facts['area_land'] + test_facts['area_water'] # area = land + water
test_facts.loc[(test_facts['area'] == 0) & (
    test_facts['area_land'] != 0) & (test_facts['area_water'] == 0), 'area'] = test_facts['area_land'] # area = land
test_facts.loc[(test_facts['area'] == 0) & (test_facts['area_land'] == 0) & (
    test_facts['area_water'] != 0), 'area'] = test_facts['area_water']  # area = water
test_facts.loc[(test_facts['area'] != 0) & (test_facts['area_land'] == 0) & (
    test_facts['area_water'] != 0) & (test_facts['area'] > test_facts['area_water']), 'area_land'] = test_facts['area'] - test_facts['area_water']  # land = area - water
test_facts.loc[(test_facts['area'] != 0) & (test_facts['area_land'] != 0) & (
    test_facts['area_water'] == 0) & (test_facts['area'] > test_facts['area_land']), 'area_water'] = test_facts['area'] - test_facts['area_land'] # water = area - land

### Checking again after fixing the values

In [19]:
test_facts.drop(columns=['area_test', 'test'], inplace=True)
# test_facts = test_facts.fillna(0.0)
test_facts['area_test'] = test_facts['area_land'] + test_facts['area_water']
test_facts['test'] = test_facts['area_test'] == test_facts['area']
print(test_facts['test'].value_counts())

test
True     256
False      5
Name: count, dtype: int64


### Well I cant do much about that unless i look for sources online... so on to the next part

In [20]:
print(test_facts[test_facts['test'] == False])

      id code            name       area  area_land  area_water   population  \
161  162   od     South Sudan   644329.0        0.0         0.0   12042910.0   
164  165   su           Sudan  1861484.0        0.0         0.0   36108853.0   
196  197   ee  European Union  4324782.0        0.0         0.0  514000000.0   
224  225   ax        Akrotiri      123.0        0.0         0.0      15700.0   
230  231   dx        Dhekelia      130.0        0.0         0.0      15700.0   

     population_growth  birth_rate  death_rate  migration_rate  area_test  \
161               4.02       36.91        8.18           11.47        0.0   
164               1.72       29.19        7.66            4.29        0.0   
196               0.25       10.20       10.20            2.50        0.0   
224               0.00        0.00        0.00            0.00        0.0   
230               0.00        0.00        0.00            0.00        0.0   

      test  
161  False  
164  False  
196  False  
224 

In [21]:
df_facts.update(test_facts)
print(df_facts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 261 non-null    int64  
 1   code               261 non-null    object 
 2   name               261 non-null    object 
 3   area               261 non-null    float64
 4   area_land          261 non-null    float64
 5   area_water         261 non-null    float64
 6   population         261 non-null    float64
 7   population_growth  261 non-null    float64
 8   birth_rate         261 non-null    float64
 9   death_rate         261 non-null    float64
 10  migration_rate     261 non-null    float64
dtypes: float64(8), int64(1), object(2)
memory usage: 22.6+ KB
None


### Fact checking population size, cities against facts sheet

In [22]:
cities_groupby = df_cities.groupby('facts_id', as_index=False).agg({'population': 'sum'})
print(cities_groupby.head())

   facts_id  population
0         1     3097000
1         2      419000
2         3     3699000
3         4       23000
4         5     6166000


### This shows that likely not every city is in the cities sheet. We need to take note of this
##### Need to consider what percentage of population is accounted for, for the data to be relevant and trustworthy

In [23]:
for i in range(1, len(df_facts)): 
    try: 
        country_population = df_facts.loc[(df_facts['id'] == i), 'population']
        cities_population = cities_groupby.loc[(cities_groupby['facts_id'] == i), 'population']
        if (country_population.equals(cities_population)):
            cities_groupby.drop(i, inplace=True)
    except:
        continue
cities_groupby

,facts_id,population
0,1,3097000
1,2,419000
2,3,3699000
3,4,23000
4,5,6166000
...,...,...
205,243,169000
206,245,56000
207,246,2475000
208,247,60000


### How it looks like in excel, at this point if you would like to take a look

In [24]:
with pd.ExcelWriter('cleaned_countries__cities_dataset.xlsx') as writer:
    # Write df_facts to the first sheet
    df_facts.to_excel(writer, sheet_name='Facts', index=False)

    # Write df_cities to the second sheet
    df_cities.to_excel(writer, sheet_name='Cities', index=False)

In [25]:
df_new = pd.merge(df_facts, df_cities, how='left', left_on='id', right_on='facts_id')
df_new = df_new.rename(columns={
    'id_x': 'country_id',
    'name_x': 'country_name',
    'population_x': 'country_population',
    'id_y': 'city_id',
    'name_y': 'city_name',
    'population_y': 'city_population'
})

df_new.to_excel('merged_countries_cities_data.xlsx', index=False)

### The merge gives me a master sheet where it shows which cities are in which countries. This sheet (to me) is ideal for visualisation on either Tableau or PowerBi, reason being by having a master sheet with all details, I can freely manipulate and extract only the data that I need in this one master sheet. For example:
1. Which capital has the highest population/gdp? (Normalise GDP by population, gdp per capita)
2. Which country has the highest gdp (aggregration)
3. For each country, does the capital has the highest number of population?
4. Birth_rate and death_rate and migration_rate to population ratio

### Machine learning wise:
1. Does a bigger population lead to higher birth_rate and death_rate?
2. Effects of birth_rate, death_rate and population and population_growth
3. If size of population is affected by area, and if so how much does ratio of area_land and area_water affects this population. Is there a sweet spot?
4. Does size of population affect GDP
6. Identify clusters of countries with similar demographic and economic profile
7. Analyze the relationship between GDP and the number of cities, potentially indicating economic diversification.